In [1]:
# import dependecies 

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

#Config should contain database username as username and database password as password
import config

In [ ]:
#Make sure the following are present in your config file:
#config.username
#config.password
#config.database_name

In [2]:
# read vaccinations csv

csv_file = "Resources/vaccinations.csv"
vaccinations_df = pd.read_csv(csv_file)
vaccinations_df.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0


In [3]:
# set columns for first table

vaccination_rate_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'daily_vaccinations' ]].copy()
vaccination_rate_df.head()

,location,iso_code,date,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations
0,Afghanistan,AFG,2021-02-22,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,1367.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,1367.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,1367.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,1367.0


In [4]:
# set columns for second table

vaccination_daily_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'daily_vaccinations_per_million' ]].copy()
vaccination_daily_df.head()

,location,iso_code,date,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,34.0


In [5]:
# group by country for third table 

# vaccination_country_df = vaccinations_df.groupby
# vaccination_country_df.head()# Setting Up Country Reference Table
countries_df = pd.DataFrame({"location": vaccinations_df["location"].unique(), "iso_code": vaccinations_df["iso_code"].unique()})
countries_df.head()

,location,iso_code
0,Afghanistan,AFG
1,Africa,OWID_AFR
2,Albania,ALB
3,Algeria,DZA
4,Andorra,AND


## Region Date Metadata

In [ ]:
# read region_date_metadata.csv

path = "Resources/region_date_metadata.csv"
region_date_df = pd.read_csv(path)
region_date_df.head()

In [ ]:
#Set column for 4th table
region_date_metadata = region_date_df[['Country_Region', 'Province_State', 'Date', 'Recoveries']].copy()
region_date_metadata.head()

In [ ]:
#Groupby the data  as per Country_region and Date
region_date_metadata_groupby = region_date_metadata.groupby(['Country_Region', 'Date']).sum()['Recoveries']
region_date_metadata_groupby = pd.DataFrame(region_date_metadata_groupby)
region_date_metadata_groupby

In [ ]:
#Removing index columns
region_date_metadata_df = region_date_metadata_groupby.reset_index(['Country_Region', 'Date'])
region_date_metadata_df

In [ ]:
# load csv to pandas dataframe

path2 = "Resources/region_metadata.csv"
region_metadata_df = pd.read_csv(path2)
region_metadata_df.head()

In [ ]:
# connect to SQL database 

protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [ ]:
# check tables

insp.get_table_names()

In [ ]:
# Push countries lookup table to database
countries_df.to_sql(name="countries",con=engine,if_exists="fail", index=False)

In [ ]:
# push first table to database  

vaccination_rate_df.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)

In [ ]:
# push second table to database  

vaccination_daily_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# push thrid table to database  

# vaccination_country_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# check data upload first table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload second table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload third table

pd.read_sql_query('select * from ', con=engine).head()